In [1]:
print("hello")

hello


In [2]:
import pandas as pd

# Adjust the path and filename as per your extracted data location
df = pd.read_csv('tourism_dataset_5000.csv')

# Show first few rows to check
df.head()


,Tourist ID,Age,Interests,Preferred Tour Duration,Accessibility,Site Name,Sites Visited,Tour Duration,Route ID,Tourist Rating,System Response Time,Recommendation Accuracy,VR Experience Quality,Satisfaction
0,1,48,"['Architecture', 'Art', 'History']",5,False,Eiffel Tower,"['Eiffel Tower', 'Great Wall of China', 'Taj M...",7,1000,1.6,3.73,97,4.5,3
1,2,37,"['Cultural', 'Nature']",6,False,Colosseum,['Great Wall of China'],1,2000,2.6,2.89,90,4.5,3
2,3,43,"['History', 'Art', 'Architecture']",6,True,Machu Picchu,['Eiffel Tower'],2,3000,1.7,2.22,94,4.7,3
3,4,46,"['Cultural', 'Art', 'Architecture']",8,False,Colosseum,"['Machu Picchu', 'Taj Mahal']",5,4000,2.0,2.34,92,4.7,3
4,5,53,"['Architecture', 'Art']",5,True,Colosseum,"['Machu Picchu', 'Taj Mahal', 'Great Wall of C...",7,5000,3.7,2.00,96,4.8,4


In [3]:
df.info()
df.describe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Tourist ID               5000 non-null   int64  
 1   Age                      5000 non-null   int64  
 2   Interests                5000 non-null   object 
 3   Preferred Tour Duration  5000 non-null   int64  
 4   Accessibility            5000 non-null   bool   
 5   Site Name                5000 non-null   object 
 6   Sites Visited            5000 non-null   object 
 7   Tour Duration            5000 non-null   int64  
 8   Route ID                 5000 non-null   int64  
 9   Tourist Rating           5000 non-null   float64
 10  System Response Time     5000 non-null   float64
 11  Recommendation Accuracy  5000 non-null   int64  
 12  VR Experience Quality    5000 non-null   float64
 13  Satisfaction             5000 non-null   int64  
dtypes: bool(1), float64(3), 

,Tourist ID,Age,Preferred Tour Duration,Tour Duration,Route ID,Tourist Rating,System Response Time,Recommendation Accuracy,VR Experience Quality,Satisfaction
count,5000.000000,5000.000000,5000.00000,5000.00000,5.000000e+03,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,2500.500000,43.886000,5.52280,3.97720,2.500500e+06,3.002740,3.238802,92.492200,4.493160,3.534200
std,1443.520003,15.268858,1.71186,1.99967,1.443520e+06,1.164353,1.016123,4.616081,0.290596,0.736029
min,1.000000,18.000000,3.00000,1.00000,1.000000e+03,1.000000,1.500000,85.000000,4.000000,3.000000
25%,1250.750000,30.000000,4.00000,2.00000,1.250750e+06,2.000000,2.360000,88.000000,4.200000,3.000000
50%,2500.500000,44.000000,6.00000,4.00000,2.500500e+06,3.000000,3.230000,92.000000,4.500000,3.000000
75%,3750.250000,57.000000,7.00000,5.00000,3.750250e+06,4.000000,4.130000,97.000000,4.700000,4.000000
max,5000.000000,70.000000,8.00000,9.00000,5.000000e+06,5.000000,5.000000,100.000000,5.000000,5.000000


In [4]:
from sklearn.preprocessing import LabelEncoder

le_interest = LabelEncoder()
df['Interests_encoded'] = le_interest.fit_transform(df['Interests'])

le_site = LabelEncoder()
df['Site_Name_encoded'] = le_site.fit_transform(df['Site Name'])



In [5]:
df['Sites_Visited_Count'] = df['Sites Visited'].apply(lambda x: len(x.split(',')))


In [6]:
features = [
    'Age', 'Preferred Tour Duration', 'Tour Duration', 'Tourist Rating', 
    'System Response Time', 'Recommendation Accuracy', 'VR Experience Quality', 
    'Satisfaction', 'Accessibility', 'Interests_encoded', 'Site_Name_encoded', 'Sites_Visited_Count'
]

X = df[features]


In [7]:
from sklearn.ensemble import IsolationForest

# Initialize the model, you can tweak hyperparameters later
model = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)


In [8]:
model.fit(X)


IsolationForest(contamination=0.05, random_state=42)

In [9]:
# Predict anomaly labels: -1 means anomaly, 1 means normal
df['anomaly'] = model.predict(X)

# Also get anomaly scores (the lower, the more anomalous)
df['anomaly_score'] = model.decision_function(X)


In [ ]:
print("Number of anomalies found:", (df['anomaly'] == -1).sum())


Number of anomalies found: 250


In [11]:
df[df['anomaly'] == -1].head()


,Tourist ID,Age,Interests,Preferred Tour Duration,Accessibility,Site Name,Sites Visited,Tour Duration,Route ID,Tourist Rating,System Response Time,Recommendation Accuracy,VR Experience Quality,Satisfaction,Interests_encoded,Site_Name_encoded,Sites_Visited_Count,anomaly,anomaly_score
33,34,44,"['Art', 'Cultural']",5,True,Colosseum,['Taj Mahal'],1,34000,4.9,4.43,88,4.2,5,24,0,1,-1,-0.003973
73,74,26,"['Architecture', 'Art']",4,True,Eiffel Tower,"['Colosseum', 'Taj Mahal', 'Great Wall of China']",5,74000,5.0,3.16,90,4.9,5,3,1,3,-1,-0.001024
108,109,66,['Cultural'],6,False,Taj Mahal,"['Great Wall of China', 'Machu Picchu', 'Eiffe...",5,109000,4.8,1.68,94,4.9,5,50,4,3,-1,-0.000158
115,116,59,"['Art', 'Nature']",3,True,Colosseum,['Colosseum'],1,116000,1.3,4.03,86,4.1,3,32,0,1,-1,-0.000374
117,118,39,"['Architecture', 'Cultural', 'History']",3,True,Machu Picchu,"['Great Wall of China', 'Eiffel Tower', 'Taj M...",5,118000,4.8,3.63,88,4.3,5,5,3,3,-1,-0.003533


What this tells about the area:
	•	Unusual Tourist Behavior: The flagged anomalies represent tourists whose visit patterns (sites visited, durations, ratings, etc.) differ notably from the majority. For example, tourists who might have:
	•	Extremely short or long tours compared to their preferences.
	•	Skipped popular sites or visited rare ones.
	•	Very low satisfaction despite typical routes.
	•	Unexpectedly high or low system interaction metrics.
	•	Potential Issues or Opportunities:
	•	Areas with many anomalies might indicate places where tourists face problems (accessibility challenges, dissatisfaction).
	•	It could also highlight hidden gems or lesser-known paths that attract a different visitor profile.
	•	Outliers might signal distress or deviations worth investigating (like sudden drop-offs or prolonged inactivity).
	•	Tourist Segmentation Insight: The model helps identify groups of tourists whose preferences or experiences do not fit usual patterns, valuable for personalized service or targeted improvements.

Conclusions You Can Draw
	•	Safety & Distress Detection: Flag tourists who appear lost, inactive unusually long, or deviate from planned safe routes.
	•	Personalized Recommendations: Detect if a tourist is deviating due to dislike/interests and suggest alternative experiences.
	•	Area Popularity & Problem Spots: Identify which locations cause frequent anomalies, indicating attractiveness or issues (access, satisfaction).
	•	Behavioral Insights: Understand diverse tourist behaviors to improve infrastructure, support, and marketing.
Representation in the App
	•	Visual Maps: Show heatmaps or pins highlighting normal vs. anomalous tourist behaviors in regions or sites.
	•	Dashboard Analytics: Present statistics on anomalies detected, types, and affected areas for tourism managers.
	•	User Notifications: Provide clear, friendly messages or guidance to tourists when anomalies (like inactivity) are detected.
	•	Reports & Alerts: Backend generates reports or real-time alerts with anomaly details to administrators or local operators.